# Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import xgboost as xgb
import lightgbm as lgb

from datetime import datetime

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor, StackingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn import svm

from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Data Loading

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
train

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0
...,...,...,...,...,...,...,...,...,...,...
7995,9993,1,9,6,14,0,0,18513,18201,1
7996,9994,0,3,10,4,0,0,17381,19797,0
7997,9995,1,9,3,9,2,1,18274,18491,1
7998,9997,0,5,10,6,1,1,18274,18491,1


In [4]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472
...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256
1996,9980,0,4,4,4,0,0,17409,17256
1997,9983,1,6,3,3,1,1,18513,18201
1998,9996,1,10,9,9,1,1,18513,18201


# Feature

- 各特長量の説明

| ヘッダ名称            | データ型 | 説明                                                       |
|---------------------|-------|----------------------------------------------------------|
| gameId             | int   | ゲームID                                                    |
| blueFirstBlood      | int   | ゲームの最初のキル。青チームが最初のキルを行った場合は1、それ以外の場合は0 |
| blueKills           | int   | 青チームによって殺された敵の数                                     |
| blueDeaths          | int   | 青チームの死亡者数                                            |
| blueAssists         | int   | 青チームのキルアシストの数                                      |
| blueEliteMonsters   | int   | 青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）       |
| blueDragons         | int   | 青チームによって殺されたドラゴンの数                               |
| blueTotalGold       | int   | 青チームの得たゴールド合計                                       |
| blueTotalExperience | int   | 青チームの得た経験値合計                                       |
| blueWins            | int   | 目的変数（青チームが勝った場合は1、それ以外の場合は0。）                |


# Feature Engineering

- Add new feature

In [5]:
def create_features(df):
    """
    与えられたデータフレームに対して特徴量エンジニアリングを行う関数。

    Args:
    df (DataFrame): 特徴量エンジニアリングを行うデータフレーム

    Returns:
    DataFrame: 新しい特徴量が追加されたデータフレーム
    """
    # キルとデスの比率
    df['killDeathRatio'] = df['blueKills'] / (df['blueDeaths'] + 1)  # ゼロ除算を防ぐために+1

    # キルとデスの差
    df['killDeathDiff'] = df['blueKills'] - df['blueDeaths']
    # 負の値を0で置き換え
    df['killDeathDiff'] = df['killDeathDiff'].clip(lower=0)

    # アシストの割合
    df['assistRate'] = df['blueAssists'] / (df['blueKills'] + 1)

    # ヘラルドの討伐数
    df['blueHerald'] = df['blueEliteMonsters'] - df['blueDragons']
    # blueHerald列の負の値を0で置き換え
    df['blueHerald'] = df['blueHerald'].clip(lower=0)

    # エリートモンスターとドラゴンの比率
    df['eliteMonsterDragonRatio'] = df['blueEliteMonsters'] / (df['blueDragons'] + 1)
    # エリートモンスターとヘラルドの比率
    df['eliteMonsterHeraldRatio'] = df['blueEliteMonsters'] / (df['blueHerald'] + 1)

    # ゴールドと経験値の対数変換
    df['logTotalGold'] = np.log1p(df['blueTotalGold'])
    df['logTotalExperience'] = np.log1p(df['blueTotalExperience'])

    # ゴールドと経験値の相互作用
    df['goldXexperience'] = df['blueTotalGold'] * df['blueTotalExperience']

    # 初期キルの影響
    df['firstBloodKillRatio'] = df['blueFirstBlood'] * df['blueKills']
    
    # キル、デス、アシストの合計
    df['totalCombatPoints'] = df['blueKills'] + df['blueDeaths'] + df['blueAssists']

    # キル・デス・アシスト比率
    total_combat_actions = df['blueKills'] + df['blueDeaths'] + df['blueAssists']
    df['killRatio'] = df['blueKills'] / total_combat_actions
    df['deathRatio'] = df['blueDeaths'] / total_combat_actions
    df['assistRatio'] = df['blueAssists'] / total_combat_actions

    # 効率的なリソース利用
    df['goldPerKill'] = df['blueTotalGold'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1
    df['experiencePerKill'] = df['blueTotalExperience'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1

    # エリートモンスターの討伐効率
    df['eliteMonstersPerGold'] = df['blueEliteMonsters'] / df['blueTotalGold']
    df['eliteMonstersPerExperience'] = df['blueEliteMonsters'] / df['blueTotalExperience']

    # チームの総合力
    df['teamStrength'] = df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']

    return df

- Apply data

In [6]:
train = create_features(train)
test = create_features(test)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   gameId                      8000 non-null   int64  
 1   blueFirstBlood              8000 non-null   int64  
 2   blueKills                   8000 non-null   int64  
 3   blueDeaths                  8000 non-null   int64  
 4   blueAssists                 8000 non-null   int64  
 5   blueEliteMonsters           8000 non-null   int64  
 6   blueDragons                 8000 non-null   int64  
 7   blueTotalGold               8000 non-null   int64  
 8   blueTotalExperience         8000 non-null   int64  
 9   blueWins                    8000 non-null   int64  
 10  killDeathRatio              8000 non-null   float64
 11  killDeathDiff               8000 non-null   int64  
 12  assistRate                  8000 non-null   float64
 13  blueHerald                  8000 

# 特徴量の効き

In [8]:
# 特徴量間の相関を観察
plt.figure(figsize=(12, 10))
sns.heatmap(train.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# model

- data

In [ ]:
# データの準備
X = train.drop(['blueWins', 'gameId'], axis=1)
y = train['blueWins']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# test_drop_gameId = test.drop('gameId', axis=1)


test_drop_gameId_temp = test.drop('gameId', axis=1)

# 特徴量の標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)  # 検証データも同じスケーラーを使用して標準化

# テストデータも同じスケーラーを使用して標準化
test_drop_gameId_scaled = scaler.transform(test_drop_gameId_temp)

# NumPy配列からPandasデータフレームに変換
X_train = pd.DataFrame(X_train, columns=X.columns)
X_val = pd.DataFrame(X_val, columns=X.columns)
test_drop_gameId = pd.DataFrame(test_drop_gameId_scaled, columns=test_drop_gameId_temp.columns)



- optuna_Lgbm

In [ ]:
def objective(trial):
    # Optunaで調整するハイパーパラメータ
    param = {
        "objective": "binary",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "max_bin": trial.suggest_int("max_bin", 200, 400),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 0.9),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 0.9),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # クロスバリデーションでのスコア計算
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    scores = []
    for tr_idx, va_idx in kf.split(X_train):
        tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]

        lgb_train = lgb.Dataset(tr_x, tr_y)
        lgb_eval = lgb.Dataset(va_x, va_y, reference=lgb_train)

        callbacks = [lgb.early_stopping(100)]
        model = lgb.train(param, lgb_train, num_boost_round=5000, callbacks=callbacks, valid_sets=lgb_eval)
        preds = model.predict(va_x)
        pred_labels = np.rint(preds)
        score = accuracy_score(va_y, pred_labels)
        scores.append(score)

    return np.mean(scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# 最適なパラメータを取得
best_params_lgbm = study.best_params
print("Best params:", best_params_lgbm)

[I 2024-01-22 09:33:32,251] A new study created in memory with name: no-name-ba2b58e7-1447-4f51-a825-be0d5a1e06b6


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[53]	valid_0's binary_logloss: 0.463162
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[38]	valid_0's binary_logloss: 0.459567
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:33:32,969] Trial 0 finished with value: 0.77125 and parameters: {'num_leaves': 90, 'max_bin': 308, 'learning_rate': 0.09215175919652555, 'bagging_fraction': 0.7367088015453607, 'bagging_freq': 6, 'feature_fraction': 0.6814651558848375, 'min_child_samples': 62}. Best is trial 0 with value: 0.77125.


Early stopping, best iteration is:
[31]	valid_0's binary_logloss: 0.481819
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[50]	valid_0's binary_logloss: 0.469906
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[113]	valid_0's binary_logloss: 0.458152
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[130]	valid_0's binary_logloss: 0.452305
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:33:33,547] Trial 1 finished with value: 0.77625 and parameters: {'num_leaves': 25, 'max_bin': 399, 'learning_rate': 0.04033851026255956, 'bagging_fraction': 0.7966433520767895, 'bagging_freq': 7, 'feature_fraction': 0.4310921609198508, 'min_child_samples': 52}. Best is trial 1 with value: 0.77625.


...

Early stopping, best iteration is:
[60]	valid_0's binary_logloss: 0.479443
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[550]	valid_0's binary_logloss: 0.452843
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[399]	valid_0's binary_logloss: 0.453653
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[346]	valid_0's binary_logloss: 0.469194
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:35:18,796] Trial 98 finished with value: 0.7773437499999999 and parameters: {'num_leaves': 8, 'max_bin': 237, 'learning_rate': 0.03216342399750909, 'bagging_fraction': 0.8708787513094256, 'bagging_freq': 1, 'feature_fraction': 0.5495461171054253, 'min_child_samples': 25}. Best is trial 10 with value: 0.77984375.


Early stopping, best iteration is:
[494]	valid_0's binary_logloss: 0.460732
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[106]	valid_0's binary_logloss: 0.471072
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[102]	valid_0's binary_logloss: 0.468586
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[92]	valid_0's binary_logloss: 0.481695
Training until validation scores don't improve for 100 rounds


[I 2024-01-22 09:35:21,099] Trial 99 finished with value: 0.77296875 and parameters: {'num_leaves': 224, 'max_bin': 210, 'learning_rate': 0.02505576624451638, 'bagging_fraction': 0.8437459211055983, 'bagging_freq': 2, 'feature_fraction': 0.5261931077436715, 'min_child_samples': 20}. Best is trial 10 with value: 0.77984375.


Early stopping, best iteration is:
[98]	valid_0's binary_logloss: 0.475329
Best params: {'num_leaves': 19, 'max_bin': 391, 'learning_rate': 0.04624302110201891, 'bagging_fraction': 0.8980745162998647, 'bagging_freq': 1, 'feature_fraction': 0.4221422845126958, 'min_child_samples': 19}


- optuna_rf

In [ ]:
def objective_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 2, 32),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 150),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 60)
    }

    clf = RandomForestClassifier(**params)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_random_forest, n_trials=100)
best_params_rf = study_rf.best_params

[I 2024-01-22 09:35:21,127] A new study created in memory with name: no-name-ccff654d-37d5-4c75-8cc4-95462cdee646
[I 2024-01-22 09:35:24,818] Trial 0 finished with value: 0.7626566533870026 and parameters: {'n_estimators': 212, 'max_depth': 18, 'min_samples_split': 98, 'min_samples_leaf': 57}. Best is trial 0 with value: 0.7626566533870026.
[I 2024-01-22 09:35:26,305] Trial 1 finished with value: 0.7589068728961722 and parameters: {'n_estimators': 87, 'max_depth': 4, 'min_samples_split': 84, 'min_samples_leaf': 26}. Best is trial 0 with value: 0.7626566533870026.
[I 2024-01-22 09:35:27,702] Trial 2 finished with value: 0.7709360486123286 and parameters: {'n_estimators': 58, 'max_depth': 32, 'min_samples_split': 96, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.7709360486123286.
[I 2024-01-22 09:35:29,006] Trial 3 finished with value: 0.7626567998485002 and parameters: {'n_estimators': 257, 'max_depth': 25, 'min_samples_split': 117, 'min_samples_leaf': 43}. Best is trial 2 with 

- optuna_SVM

In [ ]:
# 1時間でも終わらないので飛ばす


# def objective_svm(trial):
#     params = {
#         "C": trial.suggest_float("C", 0.1, 10.0),
#         "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
#         "gamma": trial.suggest_categorical("gamma", ["scale", "auto"])
#     }

#     clf = svm.SVC(**params)
#     return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

# study_svm = optuna.create_study(direction="maximize")
# study_svm.optimize(objective_svm, n_trials=100)
# best_params_svm = study_svm.best_params

- optuna_LR

In [ ]:
def objective_logistic_regression(trial):
    params = {
        "C": trial.suggest_loguniform("C", 1e-4, 10.0),
        "penalty": trial.suggest_categorical("penalty", ["l2", "none"])
    }

    clf = LogisticRegression(**params)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_logistic_regression, n_trials=100)
best_params_lr = study_lr.best_params

[I 2024-01-22 09:37:44,820] A new study created in memory with name: no-name-bf4e88fa-f2f7-4d38-bccf-dd38d4b66bee
C:\Users\104161\AppData\Local\Temp\ipykernel_14108\3235143177.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "C": trial.suggest_loguniform("C", 1e-4, 10.0),
[I 2024-01-22 09:37:45,008] Trial 0 finished with value: 0.7509372437381486 and parameters: {'C': 0.5056689450858173, 'penalty': 'none'}. Best is trial 0 with value: 0.7509372437381486.
C:\Users\104161\AppData\Local\Temp\ipykernel_14108\3235143177.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "C": trial.suggest_loguniform("C", 1e-4, 10.0),
[I 2024-01-22 09:37:45,144] Trial 1 finished with valu

- optuna_catboost

In [ ]:
def objective_catboost(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 50, 300),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "random_strength": trial.suggest_int("random_strength", 0, 100),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"])
    }

    clf = CatBoostClassifier(**params, verbose=0)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_cb = optuna.create_study(direction="maximize")
study_cb.optimize(objective_catboost, n_trials=100)
best_params_cb = study_cb.best_params


[I 2024-01-22 09:38:02,820] A new study created in memory with name: no-name-96515389-6397-4c97-9d2c-5bb8e0af92f6
[I 2024-01-22 09:38:05,690] Trial 0 finished with value: 0.7535916386888591 and parameters: {'iterations': 122, 'depth': 9, 'learning_rate': 0.2712936683624893, 'random_strength': 58, 'bagging_temperature': 0.2332758685537728, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.7535916386888591.
[I 2024-01-22 09:38:08,482] Trial 1 finished with value: 0.7724990857141015 and parameters: {'iterations': 150, 'depth': 9, 'learning_rate': 0.09267764674812431, 'random_strength': 16, 'bagging_temperature': 0.7373055683919648, 'od_type': 'Iter'}. Best is trial 1 with value: 0.7724990857141015.
[I 2024-01-22 09:38:10,825] Trial 2 finished with value: 0.7649977671944699 and parameters: {'iterations': 249, 'depth': 6, 'learning_rate': 0.15248474979101198, 'random_strength': 83, 'bagging_temperature': 0.37247049389632314, 'od_type': 'Iter'}. Best is trial 1 with value: 0.772499085714

- optuna_XGBoost

In [ ]:
def objective_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 2, 25),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 1.0)
    }

    clf = XGBClassifier(**params)
    return cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3).mean()

study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_xgboost, n_trials=100)
best_params_xgb = study_xgb.best_params

[I 2024-01-22 09:40:29,515] A new study created in memory with name: no-name-e5ea06a9-1387-4aaa-996d-a416c70295cf
[I 2024-01-22 09:40:30,111] Trial 0 finished with value: 0.7695297985436748 and parameters: {'n_estimators': 172, 'max_depth': 5, 'learning_rate': 0.10195998026383049, 'subsample': 0.9174946769577115, 'colsample_bytree': 0.6642008344085357, 'gamma': 0.9501520861347725}. Best is trial 0 with value: 0.7695297985436748.
[I 2024-01-22 09:40:31,255] Trial 1 finished with value: 0.768904554410666 and parameters: {'n_estimators': 87, 'max_depth': 21, 'learning_rate': 0.017366671198445014, 'subsample': 0.7994401039600036, 'colsample_bytree': 0.5287436288469176, 'gamma': 0.4027498494614197}. Best is trial 0 with value: 0.7695297985436748.
[I 2024-01-22 09:40:31,997] Trial 2 finished with value: 0.7523410039613441 and parameters: {'n_estimators': 259, 'max_depth': 14, 'learning_rate': 0.19241064962904772, 'subsample': 0.7611688538095949, 'colsample_bytree': 0.6286662309525248, 'gamma

- study

In [ ]:
def predict(X_train, y_train, X_test, mode, X_val):
    preds = []  # バリデーションデータに対する予測結果を格納するリスト
    preds_val = []
    preds_test = []  # テストデータに対する予測結果を格納するリスト

    # クロスバリデーションで予測を実行
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    for tr_idx, va_idx in kf.split(X_train):
        tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]

        # モデルを構築
        if mode == 'LightGBM':
            lgb_train = lgb.Dataset(tr_x, tr_y)
            lgb_eval = lgb.Dataset(va_x, va_y, reference=lgb_train)
            # 早期停止コールバックを設定
            callbacks = [lgb.early_stopping(100)]
            model = lgb.train(best_params_lgbm, lgb_train, num_boost_round=5000, callbacks=callbacks, valid_sets=lgb_eval)

        elif mode == 'RandomForest':
            model = RandomForestClassifier(random_state=123, **best_params_rf)
            model = model.fit(tr_x, tr_y)

        elif mode == 'SVM':
            model = svm.LinearSVC()
            model = model.fit(tr_x, tr_y)

        elif mode == 'LogisticRegression':
            model = LogisticRegression(**best_params_lr)
            model = model.fit(tr_x, tr_y)

        elif mode == 'CatBoost':
            model = CatBoostClassifier(**best_params_cb)
            model = model.fit(tr_x, tr_y)

        elif mode == 'XGBoost':
            model = xgb.XGBClassifier(**best_params_xgb)
            model = model.fit(tr_x, tr_y)

        # バリデーションデータに対する予測
        pred = model.predict(va_x)
        preds.append(pred)

        pred_val = model.predict(X_val)
        preds_val.append(pred_val)

        # テストデータに対する予測
        pred_test = model.predict(X_test)
        preds_test.append(pred_test)

    # バリデーションデータに対する予測結果を連結、元の順序に戻す
    preds = np.concatenate(preds, axis=0)

    # valに対する平均予測値を取得
    preds_val = np.mean(preds_val, axis=0)

    # テストデータに対する平均予測値を取得
    preds_test = np.mean(preds_test, axis=0)

    return preds, preds_val, preds_test

- 学習

In [ ]:
# 異なるモデルのベースモデルを定義
base_models = [
    ('LightGBM', lgb.LGBMClassifier(**best_params_lgbm)),
    ('RandomForest', RandomForestClassifier(random_state=123, **best_params_rf)),
    ('LogisticRegression', LogisticRegression(**best_params_lr)),
    ('XGBoost', XGBClassifier(**best_params_xgb)),
    ('CatBoost', CatBoostClassifier(**best_params_cb))
]

# スタッキングモデルを定義（ここではロジスティック回帰を使用）
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(**best_params_lr))
# stacking_model = StackingClassifier(estimators=base_models, final_estimator=RandomForestClassifier(random_state=123))

# ベースモデルのトレーニングと予測
for model_name, model in base_models:
    preds_train, preds_val, preds_test = predict(X_train, y_train, test_drop_gameId, model_name, X_val)  # テストデータを使って予測
    # トレーニングデータの予測結果を特徴として追加
    X_train[model_name] = preds_train
    X_val[model_name] = preds_val
    test_drop_gameId[model_name] = preds_test  # テストデータの特徴量として追加

# スタッキングモデルのトレーニング
stacking_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000598 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1320
[LightGBM] [Info] Number of data points in the train set: 4800, number of used features: 27
[LightGBM] [Info] Start training from score 0.488750
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[120]	valid_0's l2: 0.147399
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000381 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1325
[LightGBM] [Info] Number of data points in the train set: 4800, number of used features: 27
[LightGBM] [Info] Start training from score 0.495000
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[110]	valid_0's l2: 0.14754
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overh

c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\1041

0:	learn: 0.6637803	total: 143ms	remaining: 33.3s
1:	learn: 0.6408058	total: 151ms	remaining: 17.4s
2:	learn: 0.6232650	total: 158ms	remaining: 12.1s
3:	learn: 0.6157006	total: 164ms	remaining: 9.4s
4:	learn: 0.5999506	total: 171ms	remaining: 7.79s
5:	learn: 0.5919100	total: 173ms	remaining: 6.55s
6:	learn: 0.5812604	total: 176ms	remaining: 5.67s
7:	learn: 0.5720398	total: 178ms	remaining: 5s
8:	learn: 0.5676246	total: 181ms	remaining: 4.5s
9:	learn: 0.5620141	total: 184ms	remaining: 4.11s
10:	learn: 0.5580595	total: 187ms	remaining: 3.78s
11:	learn: 0.5548701	total: 190ms	remaining: 3.5s
12:	learn: 0.5493443	total: 193ms	remaining: 3.26s
13:	learn: 0.5481116	total: 195ms	remaining: 3.06s
14:	learn: 0.5437637	total: 199ms	remaining: 2.89s
15:	learn: 0.5427140	total: 201ms	remaining: 2.73s
16:	learn: 0.5377684	total: 204ms	remaining: 2.59s
17:	learn: 0.5346166	total: 206ms	remaining: 2.46s
18:	learn: 0.5320648	total: 208ms	remaining: 2.35s
19:	learn: 0.5315041	total: 211ms	remaining: 2.

c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0:	learn: 0.6576898	total: 4.81ms	remaining: 1.11s
1:	learn: 0.6358612	total: 8.75ms	remaining: 1.01s
2:	learn: 0.6104847	total: 11.8ms	remaining: 901ms
3:	learn: 0.5954456	total: 14.8ms	remaining: 846ms
4:	learn: 0.5777279	total: 17.9ms	remaining: 818ms
5:	learn: 0.5715598	total: 20.7ms	remaining: 785ms
6:	learn: 0.5628269	total: 24.1ms	remaining: 779ms
7:	learn: 0.5599196	total: 27.3ms	remaining: 769ms
8:	learn: 0.5522107	total: 30.6ms	remaining: 762ms
9:	learn: 0.5451876	total: 33.6ms	remaining: 750ms
10:	learn: 0.5428328	total: 36.9ms	remaining: 744ms
11:	learn: 0.5391840	total: 39.7ms	remaining: 732ms
12:	learn: 0.5372242	total: 42.5ms	remaining: 720ms
13:	learn: 0.5337911	total: 45.7ms	remaining: 715ms
14:	learn: 0.5321768	total: 48.7ms	remaining: 707ms
15:	learn: 0.5260928	total: 51.9ms	remaining: 704ms
16:	learn: 0.5237503	total: 54.7ms	remaining: 695ms
17:	learn: 0.5219720	total: 57.7ms	remaining: 689ms
18:	learn: 0.5213503	total: 60.7ms	remaining: 684ms
19:	learn: 0.5196535	t

c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\104161\Anaconda3\envs\py310_analysis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\1041

0:	learn: 0.6616946	total: 3.69ms	remaining: 856ms
1:	learn: 0.6511685	total: 6.15ms	remaining: 711ms
2:	learn: 0.6391256	total: 8.77ms	remaining: 672ms
3:	learn: 0.6231232	total: 11.4ms	remaining: 651ms
4:	learn: 0.6056124	total: 13.8ms	remaining: 631ms
5:	learn: 0.5960542	total: 16.3ms	remaining: 616ms
6:	learn: 0.5846247	total: 19.5ms	remaining: 628ms
7:	learn: 0.5791090	total: 22.6ms	remaining: 635ms
8:	learn: 0.5688590	total: 25.2ms	remaining: 627ms
9:	learn: 0.5651939	total: 27.8ms	remaining: 621ms
10:	learn: 0.5591887	total: 30.4ms	remaining: 614ms
11:	learn: 0.5560973	total: 33.2ms	remaining: 611ms
12:	learn: 0.5544452	total: 35.5ms	remaining: 600ms
13:	learn: 0.5484874	total: 38.2ms	remaining: 598ms
14:	learn: 0.5451220	total: 41ms	remaining: 596ms
15:	learn: 0.5429902	total: 43.6ms	remaining: 591ms
16:	learn: 0.5416672	total: 46.1ms	remaining: 585ms
17:	learn: 0.5373137	total: 48.8ms	remaining: 583ms
18:	learn: 0.5356538	total: 52.1ms	remaining: 586ms
19:	learn: 0.5285104	tot

StackingClassifier(estimators=[('LightGBM',
                                LGBMClassifier(bagging_fraction=0.8980745162998647,
                                               bagging_freq=1,
                                               feature_fraction=0.4221422845126958,
                                               learning_rate=0.04624302110201891,
                                               max_bin=391,
                                               min_child_samples=19,
                                               num_leaves=19)),
                               ('RandomForest',
                                RandomForestClassifier(max_depth=23,
                                                       min_samples_leaf=20,
                                                       min_samples_split=43,
                                                       n_estimators=258,
                                                       random_state...
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=269, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('CatBoost',
                                <catboost.core.CatBoostClassifier object at 0x0000029F06636350>)],
                   final_estimator=LogisticRegression())

- y_val accuracy確認

In [ ]:
# X_valデータでモデルの精度を評価
y_val_pred = stacking_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

[LightGBM] [Warning] feature_fraction is set=0.4221422845126958, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4221422845126958
[LightGBM] [Warning] bagging_fraction is set=0.8980745162998647, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8980745162998647
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


Validation Accuracy: 0.8


- testにy_test_pred結合

In [ ]:
stacking_test_preds = stacking_model.predict(test_drop_gameId)

[LightGBM] [Warning] feature_fraction is set=0.4221422845126958, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4221422845126958
[LightGBM] [Warning] bagging_fraction is set=0.8980745162998647, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8980745162998647
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [ ]:
# 現在の日時を取得してフォーマットする
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output = f'../data/output/{current_time}_submit_cv_stacking_val{val_accuracy:.4f}.csv'

In [ ]:
submit = pd.DataFrame({
    '0': test['gameId'],
    '1': stacking_test_preds
})

# カラムヘッダー消去して出力
# ヘッダーなしでCSVファイルとして保存
submit.to_csv(output, header=False, index=False)

- 出力csvチェック

In [ ]:
check = pd.read_csv(output)

In [ ]:
check

,9,1
0,15,1
1,18,1
2,23,0
3,31,1
4,32,1
...,...,...
1994,9971,0
1995,9980,0
1996,9983,1
1997,9996,1
